# Pipeline Walkthrough 

In [27]:
import os
import numpy as np
import toolbox

import toolbox.IcesheetCHANGES as changes
import toolbox.grid_generation as grid
import toolbox.initiation.grid_construction as grid_construction
import toolbox.data_handling as data 
import toolbox.store_nc as store

## Download data
...skipping for now-- assuming data is downloaded via the download notebooks...

## Batch and Homogenize Data

In [28]:
# Print the possible data sets
print(toolbox.collection.print_collections())

Available collections: 

MEaSUREs Annual Antarctic Ice Velocity Maps V001
Short name: MEaSUREs Antarctic Annual Velocity

MEaSUREs Greenland Quarterly Ice Sheet Velocity Mosaics from SAR and Landsat V005
Short name: MEaSUREs Greenland Quarterly Velocity

MEaSUREs Greenland Monthly Ice Sheet Velocity Mosaics from SAR and Landsat, Version 5
Short name: MEaSUREs Greenland Monthly Velocity

ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height, Version 2
Short name: ATL14 Antarctic Elevation

ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height Change, Version 2
Short name: ATL15 Antarctic Elevation

ATLAS/ICESat-2 L3A Sea Ice Freeboard V006
Short name: ATL10 Sea Ice Freeboard

None


### Set options
Specify collection, project_folder, and data_folder

Data folder is where data will be downloaded and stored. An external drive is recommended. 

In [29]:
collection_key = 'MEaSUREs Greenland Monthly Velocity'
#collection_key = 'ATL15 Antarctic Elevation'

project_folder = '/Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples'
data_folder = '/Volumes/Seagate/CHANGES/data_repository/tutorial'

In [30]:
# Create the object and print some attributes
#IC = changes.AntarcticCHANGES(project_folder, data_folder) 
IC = changes.GreenlandCHANGES(project_folder, data_folder)
IC.print_attributes()
print('\n')

IC.collection_info(collection_key)

Metadata path:	 /Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples/Metadata
Icesheet name:	 Greenland
Posting:	 50 km
EPSG:		 3413


Collection ID:  C2627046644-NSIDC_ECS
Collection long name:	 MEaSUREs Greenland Monthly Ice Sheet Velocity Mosaics from SAR and Landsat, Version 5
Collection short name:	 MEaSUREs Greenland Monthly Velocity
Data type:		 Velocity
Download path:		 /Volumes/Seagate/CHANGES/data_repository/tutorial/Greenland/Velocity/MEaSUREs Greenland Monthly Velocity/Data


## Create grid 

In [31]:
# get the bounds of the ice sheet and create a grid
# save the grid as a shapefile
grid.create_grid(IC)

Velocity grid dictionary generated for 50km resolution
Shapefile saved to /Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples/Metadata/Greenland/Velocity/Grid_50km


In [20]:
print(IC.grid_dict)

{'r_00_c_00': [-652925, -3384425, -602925, -3334425], 'r_00_c_01': [-602925, -3384425, -552925, -3334425], 'r_00_c_02': [-552925, -3384425, -502925, -3334425], 'r_00_c_03': [-502925, -3384425, -452925, -3334425], 'r_00_c_04': [-452925, -3384425, -402925, -3334425], 'r_00_c_05': [-402925, -3384425, -352925, -3334425], 'r_00_c_06': [-352925, -3384425, -302925, -3334425], 'r_00_c_07': [-302925, -3384425, -252925, -3334425], 'r_00_c_08': [-252925, -3384425, -202925, -3334425], 'r_00_c_09': [-202925, -3384425, -152925, -3334425], 'r_00_c_10': [-152925, -3384425, -102925, -3334425], 'r_00_c_11': [-102925, -3384425, -52925, -3334425], 'r_00_c_12': [-52925, -3384425, -2925, -3334425], 'r_00_c_13': [-2925, -3384425, 47075, -3334425], 'r_00_c_14': [47075, -3384425, 97075, -3334425], 'r_00_c_15': [97075, -3384425, 147075, -3334425], 'r_00_c_16': [147075, -3384425, 197075, -3334425], 'r_00_c_17': [197075, -3384425, 247075, -3334425], 'r_00_c_18': [247075, -3384425, 297075, -3334425], 'r_00_c_19': 

Get all cells within the bbox from a min/max col/row

In [21]:
# Specify the grid of interest and give it a unique nickname to avoid file overwrites
IC.nickname = 'test_small_2'
min_row = 21
min_col = 8
max_row = 22
max_col = 9

In [22]:
# Store the cells of interest as an attribute of the IC object
grid_of_interest = [min_row, min_col, max_row, max_col]
grid.store_cells(IC, grid_of_interest)

Cells:  ['r_21_c_08', 'r_21_c_09', 'r_22_c_08', 'r_22_c_09']


In [23]:
# for each data source  () .. TODO: loop the following for each data sourse
    
# for each cell 
    # homogenize and store the data

for cell_ID in IC.cells:
    print('Working on '+cell_ID+' ('+str(IC.cells.index(cell_ID)+1)+' of '+str(len(IC.cells))+')')
    
    IC.region_name = cell_ID

    #output_folder =  os.path.join(IC.data_folder, 'output_files', IC.icesheet_name, IC.region_name)
    #output_file = os.path.join(output_folder, IC.short_name + "_stack.nc")
    #if os.path.exists(output_file):
    #    print("        Skipping existing file...") #: " + output_file)
    #    continue
    
    output_folder =  os.path.join(IC.data_folder, 'Output', IC.icesheet_name, IC.region_name)
    output_file = os.path.join(output_folder, IC.short_name + "_stack.nc")
    if os.path.exists(output_file):
        print("        Skipping existing file...")
        continue

    # get the bounds of the cell
    IC.extents = grid.grid_cell_ID_to_cell_bounds(cell_ID, IC.grid_dict)
    
    # create the grids in memory
    grid_construction.create_grid_from_extents(IC)

    if IC.short_name == 'ATL15 Antarctic Elevation':
        data.handle_atl(IC)
    if IC.short_name == 'MEaSUREs Greenland Monthly Velocity':
        data.handle_measures(IC)
        #store.output_data_stack_measures(IC, x, y, vx_grids, vy_grids, v_grids,ex_grids, ey_grids, e_grids, output_date_pairs, source_lists)

Working on r_21_c_08 (1 of 4)
        Skipping existing file...
Working on r_21_c_09 (2 of 4)
        Skipping existing file...
Working on r_22_c_08 (3 of 4)
        Skipping existing file...
Working on r_22_c_09 (4 of 4)
        Skipping existing file...


## Quilt Grids

In [24]:
print(IC.velocity_grid_posting)

50


In [25]:
if IC.short_name == 'ATL15 Antarctic Elevation':
    grid_variable_label = 'delta_h'
elif IC.short_name == 'MEaSUREs Greenland Monthly Velocity':
    grid_variable_label = 'V'
else:
    print('WARNING: grid variable label not set.')
no_data_value = 'nan'
grid = grid.quilt_grids_and_output(IC, grid_variable_label, no_data_value)

Resolution in quilt_grids_and_output:  50
Lowest row:  21
Highest row:  22
Lowest col:  8
Highest col:  9
Resolution in function:  50
LL bounds:  [-252925, -2334425, -202925, -2284425]
UR bounds:  [-202925, -2284425, -152925, -2234425]
Grid shape:  (96, 250, 250)
Grid[0] shape:  (250, 250)
Loading cell  r_21_c_08
File path:  /Volumes/Seagate/CHANGES/data_repository/tutorial/Output/Greenland/r_21_c_08/MEaSUREs Greenland Monthly Velocity_stack.nc
Loading cell  r_21_c_09
File path:  /Volumes/Seagate/CHANGES/data_repository/tutorial/Output/Greenland/r_21_c_09/MEaSUREs Greenland Monthly Velocity_stack.nc
Loading cell  r_22_c_08
File path:  /Volumes/Seagate/CHANGES/data_repository/tutorial/Output/Greenland/r_22_c_08/MEaSUREs Greenland Monthly Velocity_stack.nc
Loading cell  r_22_c_09
File path:  /Volumes/Seagate/CHANGES/data_repository/tutorial/Output/Greenland/r_22_c_09/MEaSUREs Greenland Monthly Velocity_stack.nc
